### Process Title and Category

In [31]:
import sys
import os

import regex as re

curr_dir = os.getcwd()

print("Current Directory:", curr_dir)

parent_dir = os.path.abspath(os.path.join(curr_dir, '..'))

print("Parent Directory:", parent_dir)

sys.path.append(parent_dir)

import json

group = 'scarpe-piatte-donna'

with open(f'{parent_dir}/zalando_catalog/donna/{group}.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

Current Directory: /home/nico/UNIPI/courses/smartapp/zalando_scripts
Parent Directory: /home/nico/UNIPI/courses/smartapp


In [32]:
for item in data:
    desc = item.get('schema_description', '')
    title = item.get('title', '')

    match3 = re.match(r'(.*?) - (.*?) - (.*?)', desc)
    
    
    if match3:
        item['title'] = match3.group(1).strip()
        item['category'] = match3.group(2).strip()
        # item['color'] = match3.group(3).strip()
    else:
        match2 = re.match(r'(.*?) - (.*)', desc)
        match_category = re.match(r'(.*?) - (.*)', title)
        if match2:
            item['title'] = match2.group(1).strip()
            if match_category:
                item['category'] = match_category.group(1).strip()
            # print(match_category.group(1).strip())
        else:
            item['title'] = None
            item['category'] = None


### Process ID extraction SKU

In [33]:
for item in data:
    url = item.get('url', '')
    match_id = re.search(r'([^-]+)-([^-]+)\.html$', url)

    if match_id:
        item['id'] = match_id.group().replace('.html', '').strip()

### Color Processing

In [34]:
import csv
import os


# --- Corrected Logic ---
first_column_elements = []

# Open the file and keep the reader/file handle within the 'with' block
with open(f'{parent_dir}/zalando_scripts/gemini_parsed_colors.csv', 'r', encoding='utf-8', newline='') as f:
    # 1. Create a CSV reader object
    csv_color = csv.reader(f)
    
    # 2. OPTIONAL: Skip the header row (if present)
    next(csv_color, None)
    
    # NOTE: No need to 'seek' if you don't print or iterate before this loop.
    # We iterate directly after creating the reader.
    
    # 3. Iterate through each row in the file
    for row in csv_color:
        if row: # Check to ensure the row is not empty
            # Access the first element (index 0) of the row
            first_element = row[0] 
            first_column_elements.append(first_element)


for i, item in enumerate(data):
    item['schema_color'] = first_column_elements[i]





### Process Image for Emedding

In [35]:
count = 0
for item in data:
    count += 1
    print(count)

    if item['images']:
        for images in item['images']:
            main_image = re.search(r'.+packshot', images)
            if main_image != None:
                break
        
        item['image_link'] = main_image.group()
            
        print(main_image)
        # item['image_link'] = item['images'][0]

1
<regex.Match object; span=(0, 141), match='https://img01.ztat.net/article/spp-media-p1/49bf830eb990481aa4b2249b70726d81/734329333d7749e4bde65a8bc2c14bdf.jpg?imwidth=200&filter=packshot'>
2
<regex.Match object; span=(0, 141), match='https://img01.ztat.net/article/spp-media-p1/a78307a43aed40748aded7ce8e51deb7/0ca113684e284795898c7384156d5094.jpg?imwidth=200&filter=packshot'>
3
<regex.Match object; span=(0, 141), match='https://img01.ztat.net/article/spp-media-p1/e1252d1fce8f48e687da83961eaa2adb/2addcd08f47d4b84a343af28eaaf20b5.jpg?imwidth=200&filter=packshot'>
4
<regex.Match object; span=(0, 141), match='https://img01.ztat.net/article/spp-media-p1/562a20c849c246ff870bcb5c954dea1f/54548ac8661c494381603440609faf90.jpg?imwidth=200&filter=packshot'>
5
<regex.Match object; span=(0, 141), match='https://img01.ztat.net/article/spp-media-p1/c1611b46019032f3a256abdaad8c3a0d/a7c6d0d72b1e4991b64274005c668839.jpg?imwidth=200&filter=packshot'>
6
<regex.Match object; span=(0, 141), match='https://im

In [36]:
with open(f'{parent_dir}/zalando_scripts/{group}.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)